# Predicting sentiment from product reviews


The goal of this first notebook is to explore logistic regression and feature engineering with existing GraphLab functions.

In this notebook you will use product review data from Amazon.com to predict whether the sentiments about a product (from its reviews) are positive or negative.

* Use SFrames to do some feature engineering
* Train a logistic regression model to predict the sentiment of product reviews.
* Inspect the weights (coefficients) of a trained logistic regression model.
* Make a prediction (both class and probability) of sentiment for a new product review.
* Given the logistic regression weights, predictors and ground truth labels, write a function to compute the **accuracy** of the model.
* Inspect the coefficients of the logistic regression model and interpret their meanings.
* Compare multiple logistic regression models.

Let's get started!
    
## Fire up GraphLab Create

In [3]:
import nltk
from nltk.corpus import wordnet

[Synset('plan.n.01'),
 Synset('program.n.02'),
 Synset('broadcast.n.02'),
 Synset('platform.n.02'),
 Synset('program.n.05'),
 Synset('course_of_study.n.01'),
 Synset('program.n.07'),
 Synset('program.n.08'),
 Synset('program.v.01'),
 Synset('program.v.02')]

In [10]:
similar = []
opposite = []

priority_words = ['amazing', 'love', 'expected','great', 'easy','worth', 'old', 'powerful', 'perfect', 'loves', 
      'well', 'able', 'broke', 'works', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return','senseless','stupid',]

for wordi in priority_words:
    for syn in wordnet.synsets(wordi):
        for l in syn.lemmas():
            similar.append(l.name())
            if l.antonyms():
                opposite.append(l.antonyms()[0].name())

print len(set(similar))
print len(set(opposite))
print len(priority_words)

440
27


Make sure you have the latest version of GraphLab Create.

In [12]:
from __future__ import division
import graphlab
import math
import string

# Data preperation

We will use a dataset consisting of baby product reviews on Amazon.com.

In [30]:
amazon_reviews = graphlab.SFrame('amazon_baby.gl/')

Now, let us see a preview of what the dataset looks like.

In [33]:
amazon_reviews[0:-1]

name,review,rating
Planetwise Flannel Wipes,"These flannel wipes areOK, but in my opinion ...",3.0
Planetwise Wipe Pouch,it came early and was notdisappointed. i love ...,5.0
Annas Dream Full Quiltwith 2 Shams ...,Very soft and comfortableand warmer than it ...,5.0
Stop Pacifier Suckingwithout tears with ...,This is a product wellworth the purchase. I ...,5.0
Stop Pacifier Suckingwithout tears with ...,All of my kids have criednon-stop when I tried to ...,5.0
Stop Pacifier Suckingwithout tears with ...,"When the Binky Fairy cameto our house, we didn't ...",5.0
A Tale of Baby's Dayswith Peter Rabbit ...,"Lovely book, it's boundtightly so you may no ...",4.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",Perfect for new parents.We were able to keep ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",A friend of mine pinnedthis product on Pinte ...,5.0
"Baby Tracker&reg; - DailyChildcare Journal, ...",This has been an easy wayfor my nanny to record ...,4.0


## Build the word count vector for each review

Let us explore a specific example of a baby product.


In [34]:
amazon_reviews[269]

{'name': 'The First Years Massaging Action Teether',
 'rating': 5.0,
 'review': 'A favorite in our house!'}

In [4]:
len(products)

183531

In [10]:
rew={}

for i in products['rating']:
    if i not in rew:
        rew[i]=1
    else:
        rew[i]+=1
print rew
        
    

{1.0: 15183, 2.0: 11310, 3.0: 16779, 4.0: 33205, 5.0: 107054}


In [8]:
len(products[products['rating']>3])


140259

In [9]:
len(products[products['rating']<3])

26493

Now, we will perform 2 simple data transformations:

1. Remove punctuation using [Python's built-in](https://docs.python.org/2/library/string.html) string functionality.
2. Transform the reviews into word-counts.

**Aside**. In this notebook, we remove all punctuations for the sake of simplicity. A smarter approach to punctuations would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See [this page](https://www.cis.upenn.edu/~treebank/tokenization.html) for an example of smart handling of punctuations.

In [38]:
def remove_punctuation(text):
    import string
    return text.translate(None, string.punctuation) 

review_without_puctuation = amazon_reviews['review'].apply(remove_punctuation)
amazon_reviews['words_cnt'] = graphlab.text_analytics.count_words(review_without_puctuation)

Now, let us explore what the sample example above looks like after these 2 transformations. Here, each entry in the **word_count** column is a dictionary where the key is the word and the value is a count of the number of times the word occurs.

In [39]:
amazon_reviews[269]['words_cnt']

{'a': 1L, 'favorite': 1L, 'house': 1L, 'in': 1L, 'our': 1L}

## Extract sentiments

We will **ignore** all reviews with *rating = 3*, since they tend to have a neutral sentiment.

In [40]:
amazon_reviews = amazon_reviews[amazon_reviews['rating'] != 3]
len(amazon_reviews)

166752

Now, we will assign reviews with a rating of 4 or higher to be *positive* reviews, while the ones with rating of 2 or lower are *negative*. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label.

In [42]:
amazon_reviews['label'] = amazon_reviews['rating'].apply(lambda rating : +1 if rating > 3 else -1)
amazon_reviews[1000]

{'label': -1L,
 'name': 'Safety 1st Deluxe 4-in-1 Bath Station',
 'rating': 2.0,
 'review': "I thought this would be a great item to buy for my second baby. However, it's too much trouble with the adjustments, it doesn't have a smooth bottom surface, and there isn't enough room (even for my 3 week old!).I would buy a regular, plain baby bathtub and an infant support.",
 'word_count': {'3': 1L,
  'a': 3L,
  'adjustments': 1L,
  'an': 1L,
  'and': 2L,
  'baby': 2L,
  'bathtub': 1L,
  'be': 1L,
  'bottom': 1L,
  'buy': 2L,
  'doesnt': 1L,
  'enough': 1L,
  'even': 1L,
  'for': 2L,
  'great': 1L,
  'have': 1L,
  'however': 1L,
  'i': 1L,
  'infant': 1L,
  'isnt': 1L,
  'it': 1L,
  'item': 1L,
  'its': 1L,
  'much': 1L,
  'my': 2L,
  'oldi': 1L,
  'plain': 1L,
  'regular': 1L,
  'room': 1L,
  'second': 1L,
  'smooth': 1L,
  'support': 1L,
  'surface': 1L,
  'the': 1L,
  'there': 1L,
  'this': 1L,
  'thought': 1L,
  'to': 1L,
  'too': 1L,
  'trouble': 1L,
  'week': 1L,
  'with': 1L,
  'would

Now, we can see that the dataset contains an extra column called **sentiment** which is either positive (+1) or negative (-1).

## Split data into training and test sets

Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. We use `seed=1` so that everyone gets the same result.

In [43]:
train_data, test_data = amazon_reviews.random_split(.8, seed=1)
print len(train_data)
print len(test_data)

133416
33336


# Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data. This model will use the column **word_count** as a feature and the column **sentiment** as the target. We will use `validation_set=None` to obtain same results as everyone else.

**Note:** This line may take 1-2 minutes.

In [44]:
sentiment_model = graphlab.logistic_classifier.create(train_data,
                                                      target = 'label',
                                                      features=['words_cnt'],
                                                      validation_set=None)

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 12       | 3.000000  | 5.257798     | 0.960964          |

| 6         | 13       | 3.000000  | 5.739577     | 0.975033          |

+-----------+----------+-----------+--------------+-------------------+

TERMINATED: Terminated due to numerical difficulties.

This model may not be ideal. To improve it, consider doing one of the following:
(a) Increasing the regularization.
(b) Standardizing the input data.
(c) Removing highly correlated features.
(d) Removing `inf` and `NaN` values in the training data.

In [45]:
sentiment_model

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 121713
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 121712

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : lbfgs
Solver iterations              : 6
Solver status                  : TERMINATED: Terminated due to numerical difficulties.
Training time (sec)            : 6.2324

Settings
--------
Log-likelihood                 : inf

Highest Positive Coefficients
-----------------------------
words_cnt[mobileupdate]        : 41.9847
words_cnt[placeid]             : 41.7354
words_cnt[labelbox]            : 41.151
words_cnt[httpwwwamazoncomreviewrhgg6qp7tdnhbrefcmcrprcmtieutf8asinb00318cla0nodeid] : 40.0454
words_cnt[knobskeeping]        : 36.2091

Lowest Negative Coeffic

**Aside**. You may get an warning to the effect of "Terminated due to numerical difficulties --- this model may not be ideal". It means that the quality metric (to be covered in Module 3) failed to improve in the last iteration of the run. The difficulty arises as the sentiment model puts too much weight on extremely rare words. A way to rectify this is to apply regularization, to be covered in Module 4. Regularization lessens the effect of extremely rare words. For the purpose of this assignment, however, please proceed with the model above.

Now that we have fitted the model, we can extract the weights (coefficients) as an SFrame as follows:

In [47]:
weights = sentiment_model.coefficients
weights.column_names()

['name', 'index', 'class', 'value', 'stderr']

There are a total of `121713` coefficients in the model. Recall from the lecture that positive weights $w_j$ correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. 

Fill in the following block of code to calculate how many *weights* are positive ( >= 0). (**Hint**: The `'value'` column in SFrame *weights* must be positive ( >= 0)).

In [48]:
num_positive_weights = len(weights[weights['value']>=0])
num_negative_weights = len(weights[weights['value']<0])

print "Number of positive weights: %s " % num_positive_weights
print "Number of negative weights: %s " % num_negative_weights

Number of positive weights: 68419 
Number of negative weights: 53294 


**Quiz question:** How many weights are >= 0?

## Making predictions with logistic regression

Now that a model is trained, we can make predictions on the **test data**. In this section, we will explore this in the context of 3 examples in the test dataset.  We refer to this set of 3 examples as the **sample_test_data**.

In [49]:
sample_test_data = test_data[10:13]
print sample_test_data['rating']
sample_test_data

[5.0, 2.0, 1.0]


Columns:
	name	str
	review	str
	rating	float
	word_count	dict
	words_cnt	dict
	label	int

Rows: 3

Data:
+-------------------------------+-------------------------------+--------+
|              name             |             review            | rating |
+-------------------------------+-------------------------------+--------+
|   Our Baby Girl Memory Book   | Absolutely love it and all... |  5.0   |
| Wall Decor Removable Decal... | Would not purchase again o... |  2.0   |
| New Style Trailing Cherry ... | Was so excited to get this... |  1.0   |
+-------------------------------+-------------------------------+--------+
+-------------------------------+-------------------------------+-------+
|           word_count          |           words_cnt           | label |
+-------------------------------+-------------------------------+-------+
| {'and': 2L, 'all': 1L, 'lo... | {'and': 2L, 'all': 1L, 'lo... |   1   |
| {'and': 1L, 'wall': 1L, 't... | {'and': 1L, 'wall': 1L, 't... |   -1  |
| {'all': 1L, 'money': 1L, '... | {'all': 1L, 'money': 1L, '... |   -1  |
+-------------------------------+-------------------------------+-------+
[3 rows x 6 columns]

Let's dig deeper into the first row of the **sample_test_data**. Here's the full review:

In [15]:
sample_test_data[0]['review']

'Absolutely love it and all of the Scripture in it.  I purchased the Baby Boy version for my grandson when he was born and my daughter-in-law was thrilled to receive the same book again.'

That review seems pretty positive.

Now, let's see what the next row of the **sample_test_data** looks like. As we could guess from the sentiment (-1), the review is quite negative.

In [16]:
sample_test_data[1]['review']

'Would not purchase again or recommend. The decals were thick almost plastic like and were coming off the wall as I was applying them! The would NOT stick! Literally stayed stuck for about 5 minutes then started peeling off.'

We will now make a **class** prediction for the **sample_test_data**. The `sentiment_model` should predict **+1** if the sentiment is positive and **-1** if the sentiment is negative. Recall from the lecture that the **score** (sometimes called **margin**) for the logistic regression model  is defined as:

$$
\mbox{score}_i = \mathbf{w}^T h(\mathbf{x}_i)
$$ 

where $h(\mathbf{x}_i)$ represents the features for example $i$.  We will write some code to obtain the **scores** using GraphLab Create. For each row, the **score** (or margin) is a number in the range **[-inf, inf]**.

In [54]:
scores = sentiment_model.predict(test_data[12:19], output_type='margin')
print scores

[-14.66846040446802, 6.949581419693416, 6.6681856138165045, 4.66618951132585, 8.74410052049741, -1.886496813768412, -2.640075035514949]


### Predicting sentiment

These scores can be used to make class predictions as follows:

$$
\hat{y} = 
\left\{
\begin{array}{ll}
      +1 & \mathbf{w}^T h(\mathbf{x}_i) > 0 \\
      -1 & \mathbf{w}^T h(\mathbf{x}_i) \leq 0 \\
\end{array} 
\right.
$$

Using scores, write code to calculate $\hat{y}$, the class predictions:

In [59]:
p_score=[]
for i in scores:
    if i <=0:
        new= -1
        p_score.append(new)
    else:
        new=+1
        p_score.append(new)
print p_score
    

[-1, 1, 1, 1, 1, -1, -1]


Run the following code to verify that the class predictions obtained by your calculations are the same as that obtained from GraphLab Create.

In [60]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(test_data[12:19])

Class predictions according to GraphLab Create:
[-1L, 1L, 1L, 1L, 1L, -1L, -1L]


**Checkpoint**: Make sure your class predictions match with the one obtained from GraphLab Create.

### Probability predictions

Recall from the lectures that we can also calculate the probability predictions from the scores using:
$$
P(y_i = +1 | \mathbf{x}_i,\mathbf{w}) = \frac{1}{1 + \exp(-\mathbf{w}^T h(\mathbf{x}_i))}.
$$

Using the variable **scores** calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range **[0, 1]**.

In [61]:
p_score=[]
for i in scores:
    new= 1/(1+math.exp(i*-1))
    p_score.append(new)
print p_score

[4.261557996657548e-07, 0.9990418823663021, 0.9987309104296935, 0.990679635874886, 0.99984062647655, 0.13164441673660124, 0.06660337066339261]


**Checkpoint**: Make sure your probability predictions match the ones obtained from GraphLab Create.

In [62]:
print "Class predictions according to GraphLab Create:" 
print sentiment_model.predict(test_data[12:19], output_type='probability')

Class predictions according to GraphLab Create:
[4.261557996657545e-07, 0.9990418823663021, 0.9987309104296935, 0.9906796358748859, 0.99984062647655, 0.13164441673660124, 0.0666033706633926]


** Quiz Question:** Of the three data points in **sample_test_data**, which one (first, second, or third) has the **lowest probability** of being classified as a positive review?

# Find the most positive (and negative) review

We now turn to examining the full test dataset, **test_data**, and use GraphLab Create to form predictions on all of the test data points for faster performance.

Using the `sentiment_model`, find the 20 reviews in the entire **test_data** with the **highest probability** of being classified as a **positive review**. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:
1.  Make probability predictions on **test_data** using the `sentiment_model`. (**Hint:** When you call `.predict` to make predictions on the test data, use option `output_type='probability'` to output the probability rather than just the most likely class.)
2.  Sort the data according to those predictions and pick the top 20. (**Hint:** You can use the `.topk` method on an SFrame to find the top k rows sorted according to the value of a specified column.)

In [63]:
test_data['predictions']=sentiment_model.predict(test_data, output_type='probability')
test_data.topk('predictions',10)

name,review,rating,word_count
Munchkin Mozart MagicCube ...,My wife and I have beenliving with these 8 t ...,4.0,"{'rating': 1L, 'all': 2L,'delightfulthere': 1L, ..."
BABYBJORN Potty Chair -Red ...,Our family is juststarting EC (elimination ...,5.0,"{'managed': 1L, 'just':3L, 'being': 3L, 'over': ..."
Safety 1st Tot-Lok FourLock Assembly ...,I have a wooden desk thatseemed to be ...,5.0,"{'saying': 1L, 'all': 1L,'over': 1L, 'anyway': ..."
Summer Infant CompleteNursery Care Kit ...,This Nursery and BathCare Set contains a ...,4.0,"{'son': 1L, 'set': 2L,'help': 1L, 'less': 1L, ..."
Leachco Snoogle TotalBody Pillow ...,I have had my Snoogle forabout 12 hours and I am ...,5.0,"{'saying': 1L, 'help':1L, 'just': 5L, ..."
"HALO SleepSack Micro-Fleece Wearable Blanket, ...",I love the Sleepsackwearable blanket! I've ...,5.0,"{'all': 1L, 'just': 2L,'food': 1L, 'over': 1L, ..."
Peg Perego Primo ViaggioCar Seat / Infant Car ...,We have been using thisseat for a year now ( ...,5.0,"{'all': 1L, 'just': 2L,'move': 1L, 'through': ..."
Capri Stroller - Red Tech,First let me say that Iwas looking for a lite ...,4.0,"{'son': 3L, 'longer': 1L,'infant': 1L, 'breaki ..."
Wizard Convertible CarSeat with LATCH in ...,My son was born big andremains in the 90th ...,5.0,"{'son': 4L, 'all': 1L,'decide': 1L, 'just': ..."
"Britax MarathonConvertible Car Seat, ...",My son began using theMarathon when he was ...,5.0,"{'infant': 1L, 'over':2L, 'underside': 1L, ..."


**Quiz Question**: Which of the following products are represented in the 20 most positive reviews? [multiple choice]


Now, let us repeat this excercise to find the "most negative reviews." Use the prediction probabilities to find the  20 reviews in the **test_data** with the **lowest probability** of being classified as a **positive review**. Repeat the same steps above but make sure you **sort in the opposite order**.

In [23]:
test_data.topk('predictions',20,reverse=True)

name,review,rating,word_count,sentiment
Jolly Jumper Arctic SneakA Peek Infant Car Seat ...,"I am a ""research-aholic""in general and have ...",5.0,"{'raining': 1L, 'all':8L, 'reviewers': 1L, ...",1
Levana Safe N'See DigitalVideo Baby Monitor with ...,This is the first reviewI have ever written out ...,1.0,"{'all': 2L, 'just': 1L,'being': 1L, 'general ...",-1
Snuza Portable BabyMovement Monitor ...,I would have given theproduct 4 stars for w ...,1.0,"{'contacted': 1L,'being': 1L, 'money': ...",-1
Fisher-Price OceanWonders Aquarium Bouncer ...,We have not had ANY luckwith Fisher-Price ...,2.0,"{'fishstarfish': 1L,'all': 5L, 'managed': ...",-1
VTech Communications Safe&amp; Sounds Full Color ...,"This is my second videomonitoring system, the ...",1.0,"{'all': 4L, 'reviewers':1L, 'beware': 1L, ...",-1
Safety 1st High-DefDigital Monitor ...,We bought this babymonitor to replace a ...,1.0,"{'all': 3L, 'being': 1L,'over': 2L, ...",-1
Chicco Cortina KeyFit 30Travel System in ...,My wife and I have usedthis system in two cars ...,1.0,"{'all': 4L, 'wrestle':1L, 'overpriced': 1L, ...",-1
Prince Lionheart WarmiesWipes Warmer ...,*****IMPORTANTUPDATE*****I went to ...,1.0,"{'less': 1L, 'move': 1L,'not': 4L, 'yes': 1L, ...",-1
Valco Baby Tri-mode TwinStroller EX- Hot ...,I give one star to thedimension: 1. being 29 ...,1.0,"{'limited': 2L, 'forget':1L, 'switched': 1L, ...",-1
Adiri BPA Free NaturalNurser Ultimate Bottle ...,I will try to write anobjective review of the ...,2.0,"{'all': 2L, 'forget': 1L,'grateful': 1L, 'morei': ...",-1


**Quiz Question**: Which of the following products are represented in the 20 most negative reviews?  [multiple choice]

## Compute accuracy of the classifier

We will now evaluate the accuracy of the trained classifer. Recall that the accuracy is given by


$$
\mbox{ Classifier Accuracy} = \frac{\mbox{Number of correctly labeled data points }}{\mbox{Number of total data points}}
$$

This can be computed as follows:

* **Step 1:** Use the trained model to compute class predictions (**Hint:** Use the `predict` method)
* **Step 2:** Count the number of data points when the predicted class labels match the ground truth labels (called `true_labels` below).
* **Step 3:** Divide the total number of correct predictions by the total number of data points in the dataset.

Complete the function below to compute the classification accuracy:

In [84]:
def get_classification_accuracy(model, data, true_labels):
    # First get the predictions
    data['predictions']=model.predict(data)
    
    # Compute the number of correctly classified examples
    matched=data[data['predictions']==true_labels]
    num_correct=len(matched)

    # Then compute accuracy by dividing num_correct by total number of examples
    ## YOUR CODE HERE
    accuracy=num_correct/len(data)*1.0000
    
    return round(accuracy,4)

Now, let's compute the classification accuracy of the **sentiment_model** on the **test_data**.

In [65]:
get_classification_accuracy(sentiment_model, test_data, test_data['label'])

0.91

In [50]:
sentiment_model.coefficients.sort('value', ascending=False).print_rows(num_rows=21)

+------------+-------------------------------+-------+---------------+--------+
|    name    |             index             | class |     value     | stderr |
+------------+-------------------------------+-------+---------------+--------+
| word_count |          mobileupdate         |   1   | 41.9846857331 |  None  |
| word_count |            placeid            |   1   | 41.7353817239 |  None  |
| word_count |            labelbox           |   1   | 41.1510061892 |  None  |
| word_count | httpwwwamazoncomreviewrhgg... |   1   | 40.0453987161 |  None  |
| word_count |          knobskeeping         |   1   | 36.2091068414 |  None  |
| word_count |          chinaseveral         |   1   | 35.6239528378 |  None  |
| word_count |           pailupdate          |   1   | 33.3530175085 |  None  |
| word_count |              073              |   1   | 33.3530175085 |  None  |
| word_count |             71613             |   1   | 33.3530175085 |  None  |
| word_count |          concurrently    

**Quiz Question**: What is the accuracy of the **sentiment_model** on the **test_data**? Round your answer to 2 decimal places (e.g. 0.76).

**Quiz Question**: Does a higher accuracy value on the **training_data** always imply that the classifier is better?

## Learn another classifier with fewer words

There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected a 20 words to work with. These are:

In [89]:
important_online_wprds = ['amazing', 'love','eureka','every','decision','great', 'easy','worth', 'old', 'powerful', 'perfect', 'loves', 
      'well', 'able', 'helped', 'broke', 'works', 'less', 'even', 'waste', 'disappointed', 'penny',
      'work', 'product', 'money', 'would', 'return']

In [90]:
len(important_online_wprds)

27

For each review, we will use the **word_count** column and trim out all words that are **not** in the **significant_words** list above. We will use the [SArray dictionary trim by keys functionality]( https://dato.com/products/create/docs/generated/graphlab.SArray.dict_trim_by_keys.html). Note that we are performing this on both the training and test set.

In [91]:
train_data['words_cnt_small'] = train_data['words_cnt'].dict_trim_by_keys(important_online_wprds, exclude=False)
test_data['words_cnt_small'] = test_data['words_cnt'].dict_trim_by_keys(important_online_wprds, exclude=False)

Let's see what the first example of the dataset looks like:

In [92]:
train_data[0]['review']

'it came early and was not disappointed. i love planet wise bags and now my wipe holder. it keps my osocozy wipes moist and does not leak. highly recommend it.'

The **word_count** column had been working with before looks like the following:

In [93]:
print train_data[1]['words_cnt']

{'and': 2L, 'quilt': 1L, 'it': 1L, 'comfortable': 1L, 'warmer': 1L, 'size': 1L, 'anyone': 1L, 'for': 1L, 'looking': 1L, 'to': 1L, 'recommend': 1L, 'type': 1L, 'full': 1L, 'very': 1L, 'perfectlywould': 1L, 'than': 1L, 'this': 1L, 'of': 1L, 'bed': 1L, 'the': 1L, 'soft': 1L, 'looksfit': 1L}


Since we are only working with a subet of these words, the column **word_count_subset** is a subset of the above dictionary. In this example, only 2 `significant words` are present in this review.

In [78]:
print train_data[1]['words_cnt_small']

{}


## Train a logistic regression model on a subset of data

We will now build a classifier with **word_count_subset** as the feature and **sentiment** as the target. 

In [94]:
simple_model = graphlab.logistic_classifier.create(train_data,
                                                   target = 'label',
                                                   features=['words_cnt_small'],
                                                   validation_set=None,l2_penalty=0.01)
simple_model

Logistic regression:

--------------------------------------------------------

Number of examples          : 133416

Number of classes           : 2

Number of feature columns   : 1

...

| 5         | 6        | 0.613781     | 0.866808          |

| 6         | 7        | 0.729696     | 0.866808          |

+-----------+----------+--------------+-------------------+

SUCCESS: Optimal solution found.

Class                          : LogisticClassifier

Schema
------
Number of coefficients         : 27
Number of examples             : 133416
Number of classes              : 2
Number of feature columns      : 1
Number of unpacked features    : 26

Hyperparameters
---------------
L1 penalty                     : 0.0
L2 penalty                     : 0.01

Training Summary
----------------
Solver                         : newton
Solver iterations              : 7
Solver status                  : SUCCESS: Optimal solution found.
Training time (sec)            : 0.8769

Settings
--------
Log-likelihood                 : 44289.5894

Highest Positive Coefficients
-----------------------------
words_cnt_small[penny]         : 2.1722
words_cnt_small[loves]         : 1.712
words_cnt_small[perfect]       : 1.543
words_cnt_small[love]          : 1.3807
words_cnt_small[helped]        : 1.3478

Lowest Negative Coefficients
----------------------------
words_cnt_small[disappointed]  : -2.3211
words

We can compute the classification accuracy using the `get_classification_accuracy` function you implemented earlier.

In [95]:
get_classification_accuracy(simple_model, test_data, test_data['label'])

0.8696

Now, we will inspect the weights (coefficients) of the **simple_model**:

In [98]:
simple_model.coefficients.sort('value', ascending=False).print_rows(num_rows=5)

+-----------------+----------+-------+------------------+-----------------+
|       name      |  index   | class |      value       |      stderr     |
+-----------------+----------+-------+------------------+-----------------+
| words_cnt_small |  penny   |   1   |  2.17215513533   |  0.287658580134 |
| words_cnt_small |  loves   |   1   |  1.71203820575   |  0.048278451783 |
| words_cnt_small | perfect  |   1   |  1.54297016019   | 0.0497713638284 |
| words_cnt_small |   love   |   1   |  1.38068629791   | 0.0303469127422 |
| words_cnt_small |  helped  |   1   |  1.34782023841   |  0.116431313863 |
|   (intercept)   |   None   |   1   |  1.31497207836   | 0.0120440546551 |
| words_cnt_small | amazing  |   1   |  1.28360377522   |  0.108076388832 |
| words_cnt_small |   easy   |   1   |  1.21342438088   | 0.0292721532809 |
| words_cnt_small |  great   |   1   |  0.933650759739  | 0.0210048523805 |
| words_cnt_small |  works   |   1   |  0.526194335348  | 0.0368850141415 |
| words_cnt_

In [100]:
coefficients = list(coefficients[1:])


NameError: name 'coefficients' is not defined

Let's sort the coefficients (in descending order) by the **value** to obtain the coefficients with the most positive effect on the sentiment.

In [97]:
simple_model.coefficients.sort('value', ascending=True).print_rows(num_rows=21)

+-----------------+--------------+-------+------------------+-----------------+
|       name      |    index     | class |      value       |      stderr     |
+-----------------+--------------+-------+------------------+-----------------+
| words_cnt_small | disappointed |   1   |  -2.32109532311  | 0.0503206582041 |
| words_cnt_small |    return    |   1   |  -2.13973107135  | 0.0579093105556 |
| words_cnt_small |    waste     |   1   |  -2.02945363126  | 0.0647650331291 |
| words_cnt_small |    broke     |   1   |  -1.66662710531  | 0.0580736169733 |
| words_cnt_small |    money     |   1   | -0.936024098701  | 0.0356036727151 |
| words_cnt_small |     work     |   1   | -0.625653749794  | 0.0229422217973 |
| words_cnt_small |     even     |   1   | -0.502099003726  | 0.0198806244262 |
| words_cnt_small |    would     |   1   | -0.345627403454  | 0.0126965833558 |
| words_cnt_small |   product    |   1   |  -0.3365718498   | 0.0155005140705 |
| words_cnt_small |     less     |   1  

**Quiz Question**: Consider the coefficients of **simple_model**. There should be 21 of them, an intercept term + one for each word in **significant_words**. How many of the 20 coefficients (corresponding to the 20 **significant_words** and *excluding the intercept term*) are positive for the `simple_model`?

In [ ]:
#answer is 11

**Quiz Question**: Are the positive words in the **simple_model** (let us call them `positive_significant_words`) also positive words in the **sentiment_model**?

# Comparing models

We will now compare the accuracy of the **sentiment_model** and the **simple_model** using the `get_classification_accuracy` method you implemented above.

First, compute the classification accuracy of the **sentiment_model** on the **train_data**:

In [85]:
get_classification_accuracy(sentiment_model, train_data, train_data['sentiment'])

RuntimeError: Runtime Exception. Column name sentiment does not exist.

Now, compute the classification accuracy of the **simple_model** on the **train_data**:

In [87]:
get_classification_accuracy(simple_model, train_data, train_data['label'])

0.8667

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TRAINING set?

Now, we will repeat this excercise on the **test_data**. Start by computing the classification accuracy of the **sentiment_model** on the **test_data**:

In [53]:
get_classification_accuracy(simple_model, test_data, test_data['sentiment'])

0.87

Next, we will compute the classification accuracy of the **simple_model** on the **test_data**:

In [54]:
get_classification_accuracy(sentiment_model, test_data, test_data['sentiment'])

0.91

**Quiz Question**: Which model (**sentiment_model** or **simple_model**) has higher accuracy on the TEST set?

## Baseline: Majority class prediction

It is quite common to use the **majority class classifier** as the a baseline (or reference) model for comparison with your classifier model. The majority classifier model predicts the majority class for all data points. At the very least, you should healthily beat the majority class classifier, otherwise, the model is (usually) pointless.

What is the majority class in the **train_data**?

In [47]:
num_positive  = (train_data['sentiment'] == +1).sum()
num_negative = (train_data['sentiment'] == -1).sum()
print num_positive
print num_negative

majority=round((num_positive/(num_positive+num_negative)),2)
print majority

112164
21252
0.84


Now compute the accuracy of the majority class classifier on **test_data**.

**Quiz Question**: Enter the accuracy of the majority class classifier model on the **test_data**. Round your answer to two decimal places (e.g. 0.76).

In [49]:
num_positive  = (test_data['sentiment'] == +1).sum()
num_negative = (test_data['sentiment'] == -1).sum()
print num_positive
print num_negative

majority=round((num_positive/(num_positive+num_negative)),2)
print majority

28095
5241
0.84


In [1]:
import numpy

ImportError: No module named 'numpy'

**Quiz Question**: Is the **sentiment_model** definitely better than the majority class classifier (the baseline)?